In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn import LSTM

data = pd.read_csv('dynamic_cleaned.csv', parse_dates=['charttime'])
data1_train_encoded = pd.read_csv('static_train_3.csv')
data1_val_encoded = pd.read_csv('static_val_3.csv')
data1_test_encoded = pd.read_csv('static_test_3.csv')

def filter_and_sort(dynamic, encoded):
    unique_ids = set(encoded['id'].unique())
    filtered = dynamic[dynamic['id'].isin(unique_ids)]
    return filtered.sort_values(by=['id', 'charttime'])

dynamic_train_sorted = filter_and_sort(data, data1_train_encoded)
dynamic_val_sorted = filter_and_sort(data, data1_val_encoded)
dynamic_test_sorted = filter_and_sort(data,data1_test_encoded)

features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']
def concatenate_features(record):
    return [record[feature] for feature in features]

concatenated_data_train = dynamic_train_sorted.apply(concatenate_features, axis=1).groupby(dynamic_train_sorted['id']).agg(list)
concatenated_data_val = dynamic_val_sorted.apply(concatenate_features, axis=1).groupby(dynamic_val_sorted['id']).agg(list)
concatenated_data_test = dynamic_test_sorted.apply(concatenate_features, axis=1).groupby(dynamic_test_sorted['id']).agg(list)

def prepare_sequences(data):
    sequences = data.tolist()
    tensor_sequences = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    return pad_sequence(tensor_sequences, batch_first=True, padding_value=0.0)

padded_tensor_sequences_train = prepare_sequences(concatenated_data_train)
padded_tensor_sequences_val = prepare_sequences(concatenated_data_val)
padded_tensor_sequences_test = prepare_sequences(concatenated_data_test)

num_features = 9
hidden_units = 16

lstm = LSTM(input_size=num_features, hidden_size=hidden_units, batch_first=True)
id_train_counts_list = dynamic_train_sorted.id.value_counts(sort=False).tolist()
id_val_counts_list = dynamic_val_sorted.id.value_counts(sort=False).tolist()
id_test_counts_list = dynamic_test_sorted.id.value_counts(sort=False).tolist()

lengths_train = id_train_counts_list
packed_input_train = pack_padded_sequence(padded_tensor_sequences_train, lengths_train, batch_first=True, enforce_sorted=False)
lengths_val = id_val_counts_list
packed_input_val = pack_padded_sequence(padded_tensor_sequences_val, lengths_val, batch_first=True, enforce_sorted=False)
length_test = id_test_counts_list
packed_input_test = pack_padded_sequence(padded_tensor_sequences_test, length_test, batch_first=True, enforce_sorted = False)

class SequenceDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = [torch.tensor(s, dtype=torch.float32) for s in sequences]
        self.targets = torch.tensor(targets, dtype=torch.long)
        self.lengths = [len(seq) for seq in sequences]  # Store lengths for dynamic batching

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx], self.lengths[idx]

def collate_fn(batch):
    batch.sort(key=lambda x: x[2], reverse=True)
    sequences, targets, lengths = zip(*batch)

    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    targets = torch.stack(targets)
    lengths = torch.tensor(lengths)

    return sequences_padded, targets, lengths

def create_loader(sequences, targets, batch_size):
    dataset = SequenceDataset(sequences, targets)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    return loader

target_series_train = data1_train_encoded[['los_icu_binned_1 to 2 days','los_icu_binned_2 to 4 days','los_icu_binned_4+ days']].groupby(data1_train_encoded['id']).agg(list)
tensor_targets_train = [torch.tensor(tgt, dtype=torch.float32) for tgt in target_series_train.values.tolist()]
tensor_targets_train = [tgt.view(1, -1) for tgt in tensor_targets_train]
target_series_val = data1_val_encoded[['los_icu_binned_1 to 2 days','los_icu_binned_2 to 4 days','los_icu_binned_4+ days']].groupby(data1_val_encoded['id']).agg(list)
tensor_targets_val = [torch.tensor(tgt, dtype=torch.float32) for tgt in target_series_val.values.tolist()]
tensor_targets_val = [tgt.view(1, -1) for tgt in tensor_targets_val]
target_series_test = data1_test_encoded[['los_icu_binned_1 to 2 days','los_icu_binned_2 to 4 days','los_icu_binned_4+ days']].groupby(data1_test_encoded['id']).agg(list)
tensor_targets_test = [torch.tensor(tgt, dtype=torch.float32) for tgt in target_series_test.values.tolist()]
tensor_targets_test = [tgt.view(1, -1) for tgt in tensor_targets_test]
tensor_targets_train = [torch.tensor(tgt, dtype=torch.float32).view(1, -1) for tgt in target_series_train.values.tolist()]
tensor_targets_val = [torch.tensor(tgt, dtype=torch.float32).view(1, -1) for tgt in target_series_val.values.tolist()]
tensor_targets_test = [torch.tensor(tgt, dtype=torch.float32).view(1, -1) for tgt in target_series_test.values.tolist()]
flat_tensor_targets_train = torch.cat(tensor_targets_train).argmax(1)
flat_tensor_targets_val = torch.cat(tensor_targets_val).argmax(1)
flat_tensor_targets_test = torch.cat(tensor_targets_test).argmax(1)


train_loader = create_loader(padded_tensor_sequences_train, flat_tensor_targets_train, batch_size=64)
val_loader = create_loader(padded_tensor_sequences_val, flat_tensor_targets_val, batch_size=64)
test_loader = create_loader(padded_tensor_sequences_test,flat_tensor_targets_test,batch_size = 64)

C:\Users\游骐榕\AppData\Local\Temp\ipykernel_8508\1678221835.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('dynamic_cleaned.csv', parse_dates=['charttime'])
C:\Users\游骐榕\AppData\Local\Temp\ipykernel_8508\1678221835.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.sequences = [torch.tensor(s, dtype=torch.float32) for s in sequences]
C:\Users\游骐榕\AppData\Local\Temp\ipykernel_8508\1678221835.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets, dtype=torch.long)


In [4]:
import torch
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score

model = lstm
# Parameters
learning_rate = 0.001
num_epochs = 50
batch_size = 64

# Optimizer and Loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs):
    class_names = ['1 to 2 days', '2 to 4 days', '4+ days']
    for epoch in range(num_epochs):
        model.train()  # Switch to training mode
        total_loss = 0
        true_labels = []
        predictions = []

        for sequences, targets, lengths in train_loader:
            optimizer.zero_grad()
            outputs, _ = model(sequences)
            outputs_last_step = outputs[:, -1, :]
            loss = criterion(outputs_last_step, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs_last_step.data, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(targets.tolist())

        avg_loss = total_loss / len(train_loader)
        train_accuracy = accuracy_score(true_labels, predictions)
        train_precision = precision_score(true_labels, predictions, average='macro')
        train_recall = recall_score(true_labels, predictions, average='macro')
        train_f1 = f1_score(true_labels, predictions, average='macro')
        print(f'Epoch {epoch+1}: Training Loss: {avg_loss}, Accuracy: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1 Score: {train_f1}')

        model.eval()  # Switch to evaluation mode
        val_loss = 0
        val_true_labels = []
        val_predictions = []

        with torch.no_grad():
            for sequences, targets, lengths in val_loader:
                outputs, _ = model(sequences)
                outputs_last_step = outputs[:, -1, :]
                loss = criterion(outputs_last_step, targets)
                val_loss += loss.item()

                _, predicted = torch.max(outputs_last_step.data, 1)
                val_predictions.extend(predicted.tolist())
                val_true_labels.extend(targets.tolist())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = accuracy_score(val_true_labels, val_predictions)
        val_precision = precision_score(val_true_labels, val_predictions, average='macro')
        val_recall = recall_score(val_true_labels, val_predictions, average='macro')
        val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
        print(f'Epoch {epoch+1}: Validation Loss: {avg_val_loss}, Accuracy: {val_accuracy}, Precision: {val_precision}, Recall: {val_recall}, F1 Score: {val_f1}')
        model.eval()
        test_loss = 0
        test_true_labels = []
        test_predictions = []

        with torch.no_grad():
            for sequences, targets, lengths in test_loader:
                outputs, _ = model(sequences)
                outputs_last_step = outputs[:, -1, :]
                loss = criterion(outputs_last_step, targets)
                test_loss += loss.item()

                _, predicted = torch.max(outputs_last_step.data, 1)
                test_predictions.extend(predicted.tolist())
                test_true_labels.extend(targets.tolist())

        avg_test_loss = test_loss / len(test_loader)
        test_accuracy = accuracy_score(test_true_labels, test_predictions)
        test_precision = precision_score(test_true_labels, test_predictions, average='macro')
        test_recall = recall_score(test_true_labels, test_predictions, average='macro')
        test_f1 = f1_score(test_true_labels, test_predictions, average='macro')
        print(f'Epoch {epoch+1}: Test Loss: {avg_test_loss}, Accuracy: {test_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1 Score: {test_f1}')
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Stopping early at epoch {epoch+1}")
                break
# Train the model
train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs)

C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

Epoch 1: Training Loss: 1.560286916670252, Accuracy: 0.3611397627643115, Precision: 0.17758338876417928, Recall: 0.24636565559412818, F1 Score: 0.20477398603893734
Epoch 1: Validation Loss: 1.5623806416988373, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 1: Test Loss: 1.5623805150389671, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847


C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

Epoch 2: Training Loss: 1.5602678324355455, Accuracy: 0.33806085611139763, Precision: 0.172336655351009, Recall: 0.24612840153726295, F1 Score: 0.15846508183353886
Epoch 2: Validation Loss: 1.5623564422130585, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 2: Test Loss: 1.562356524169445, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847


C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

...

C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

Epoch 48: Training Loss: 1.560258372885282, Accuracy: 0.3392212480660134, Precision: 0.08481624758220503, Recall: 0.2499050151975684, F1 Score: 0.12664869548474056
Epoch 48: Validation Loss: 1.562199927866459, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 48: Test Loss: 1.5621999353170395, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847


C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

Epoch 49: Training Loss: 1.560258372885282, Accuracy: 0.3392212480660134, Precision: 0.08481624758220503, Recall: 0.2499050151975684, F1 Score: 0.12664869548474056
Epoch 49: Validation Loss: 1.5621999576687813, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 49: Test Loss: 1.5621999651193619, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 50: Training Loss: 1.560258372885282, Accuracy: 0.3392212480660134, Precision: 0.08481624758220503, Recall: 0.2499050151975684, F1 Score: 0.12664869548474056
Epoch 50: Validation Loss: 1.5621999725699425, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847
Epoch 50: Test Loss: 1.5622000023722649, Accuracy: 0.3371134020618557, Precision: 0.1470435684647303, Recall: 0.24839240775239402, F1 Score: 0.12739320564084847


C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\游骐榕\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is i

In [22]:
torch.save(model, 'lstm_1cell.pth')